In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
authors = pd.read_pickle("../../data/interim/european_taxonomic_authors_no_duplicates.pkl")
# less columns and author ID as index quicken processing
authors = authors[["author_id", "author_display_name", "orcid",
                   "inst_id", "inst_display_name",  "species_subject"]]
authors = authors.set_index("author_id")

levels = ["genus", "family", "order", "class", "phylum", "kingdom"]
for level in levels:
    authors[level] = [list() for x in range(len(authors.index))]

# GBIF taxonomic backbone
backbone = pd.read_csv("../../data/external/backbone/Taxon.tsv", sep="\t", on_bad_lines='skip')

# reduce size of backbone for easier searching
backbone = backbone[backbone["taxonomicStatus"]!="doubtful"]
backbone = backbone[["canonicalName",] + levels]
# remove taxa with no known species name, genus, family,...
backbone = backbone.dropna().drop_duplicates(ignore_index=True).reset_index(drop=True)

C:\Users\melanie\AppData\Local\Temp\ipykernel_12256\2542485849.py:12: DtypeWarning: Columns (9,10,16,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  backbone = pd.read_csv("../../data/external/backbone/Taxon.tsv", sep="\t", on_bad_lines='skip')


In [8]:
# get initial first name + last name for every author
truncated_names = []

for author in authors.itertuples():
    first_initial = author.author_display_name[0]
    last_name = author.author_display_name.split(" ")[-1]
    truncated_names.append(first_initial + " " + last_name)
    
authors["truncatedName"] = truncated_names

In [3]:
# backbone to dictionary for quicker processing
seen_species = {}

for species in backbone.itertuples():
    if species.canonicalName not in seen_species:
        seen_species[species.canonicalName] = list(species)[2:]

In [4]:
# for every author, break down each species associated with them into genus, family,... etc
for i, author in authors.iterrows():
    for species in author["species_subject"]:
        if species in seen_species:
            # taxonomic levels such as genus, family
            for l, level in enumerate(levels):
                taxon_name = seen_species[species][l]
                # make sure there are no duplicates
                if taxon_name not in author[level]:
                    authors.loc[i, level].append(taxon_name)

In [ ]:
def match(a, b):
    same = False
    # if no known orders for one of them, just use institution 
    if a.order == [] or b.order == []:
        if a.inst_id == b.inst_id:
            same = True
    # if both have known orders, orders and institution must match
    else:
        if a.inst_id == b.inst_id and len(set(a.order).intersection(set(b.order))) > 0:
            same = True
    
    return same


def cluster(matches):
    clusters = []
    
    for match in matches:
        # first list
        if len(clusters) == 0:
            clusters.append(match)
        # check if it matches any existing groups
        match_with_groups = []
        for i, group in enumerate(clusters):
            if len(set(match).intersection(set(group))) > 0:
                match_with_groups.append(i)
        print(match_with

In [55]:
def match(a, b):
    same = False
    # if no known orders for one of them, just use institution 
    if a.order == [] or b.order == []:
        if a.inst_id == b.inst_id:
            same = True
    # if both have known orders, orders and institution must match
    else:
        if a.inst_id == b.inst_id and len(set(a.order).intersection(set(b.order))) > 0:
            same = True
    
    return same


def cluster(matches):
    clusters = []
    
    for match in matches:
        # check if it matches any existing groups
        match_with_groups = []
        for i, group in enumerate(clusters):
            if len(set(match).intersection(set(group))) > 0:
                match_with_groups.append(i)
        
        # if it fits with no existing group, add it by itself
        if len(match_with_groups) == 0:
            clusters.append(match)
        # if it fits with one existing group, add it to that group
        elif len(match_with_groups) == 1:
            clusters[match_with_groups[0]].extend(match)
        # if it fits with multiple groups, mash those groups together
        else:
            print(clusters) # apparently this never happens
            supergroup = []
            # remove each group and add its contents to the new supergroup
            for j in match_with_groups.sort(reverse=True):
                supergroup.extend(clusters.pop(j))
            supergroup.extend(match)
            clusters.append(supergroup)
            print(clusters)
    
    # turn into a list of sets to get unique values
    clusters = [set(x) for x in clusters] 
    return clusters

In [56]:
# emergency meeting: go over every duplicated name
true_people = []
duplicates = authors[authors.duplicated(subset=["truncatedName"], keep=False)]

for name in set(duplicates["truncatedName"]):
    # get all trund names that are exact string matches to this name
    #same_names = duplicates[duplicates["author_display_name"]==name]
    same_names = duplicates[duplicates["truncatedName"]==name]
    matches = []
    
    for person_a in same_names.itertuples():
        aliases = [person_a.Index,]
        
        for person_b in same_names.itertuples():
            if match(person_a, person_b) and person_a.Index!=person_b.Index:
                aliases.append(person_b.Index)
                
        matches.append(aliases)

    true_people.extend(cluster(matches))

In [42]:
merged_df = pd.concat(merged_people, ignore_index=True)        
merged_df.to_csv("../../data/interim/merged_people.csv")

true_authors.to_pickle("../../data/processed/european_authors_disambiguated.pkl")
true_authors.to_csv("../../data/processed/european_authors_disambiguated.tsv", sep="\t")

C:\Users\melanie\anaconda3\lib\site-packages\pandas\core\internals\construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])
